### Shared Attributes

In [8]:
import numpy as np
from unyt import unyt_array, unyt_quantity

In [102]:
class BaseParticleType:
    """BaseParticleType class that implements common methods and attributes for particle ensembles. These methods and attributes
    are accesible for all particle types and hence this class acts as a bridge between stars, darkmatter and gas, allowing 
    them to access properties of one another. This makes sense, as particles types in cosmological simulations are coupled to
    each other.
    
    It also simplifies the code, as a plethora of common methods are displaced to here.
    """
    _shared_attrs = {
        "darkmatter": {"rockstar_center": None, "rockstar_vel": None, "rvir": None, "rs": None, "c": None, 'vmax': None, 'vrms': None},
        "stars": {"ML": None},
    }


    @classmethod
    def format_value(cls, value):
        """Formats value using unyt if value != None, else returns none
        """
        if value is None:
            return None
            
        if type(value) == tuple:
            assert len(value) >= 1 and len(value) <= 2, f"Tuple must be of the formt (X,)==(X,'dimensionless') or (X,unit). Your provided {value}."
            if value[0] is None: return None
            else: return unyt_array(*value)
                
        else:
            return cls.format_value((value,))

    @classmethod
    def set_shared_attrs(cls, pt, kwargs):
        """Set class-level shared attributes for a specific particle type.
        """
        if pt not in cls._shared_attrs:
            raise ValueError(f"Unknown particle type: {pt}")
        
        for key, value in kwargs.items():
            if key in cls._shared_attrs[pt]:
                cls._shared_attrs[pt][key] = value
            else:
                raise ValueError(f"Invalid shared attribute '{key}' for type '{pt}'")

        print(cls._shared_attrs)
        return None
    
    @classmethod
    def get_shared_attr(cls, pt, key):
        """Get a specific shared attribute for a particle type.
        """
        if pt not in cls._shared_attrs:
            raise ValueError(f"Unknown particle type: {pt}")
        return cls._shared_attrs[pt].get(key)

    @classmethod
    def update_shared_attr(cls, pt, key, value):
        """Update a specific shared attribute for a particle type.
        """
        if (pt in cls._shared_attrs) and (key in cls._shared_attrs[pt]):
            cls._shared_attrs[pt][key] = value
        else:
            raise ValueError(f"Cannot update: '{key}' not valid for '{pt}'")

    @classmethod
    def list_shared_attributes(cls, pt):
        """List all shared attributes for a given particle type."""
        return list(cls._shared_attrs.get(pt, {}).keys())

In [122]:
class pty(BaseParticleType):
    def __init__(self, pt, **kwargs):
        super().__init__()
        self.ptype = pt      
        self.units = {
            'dimensionless': 'dimensionless',
            'mass': "Msun",
            'time': "Gyr",
            'length': "kpc",
            'velocity': "km/s",
            'comoving': False
        }
        self.set_shared_attrs(pt, kwargs)





    # Dynamically visible properties for darkmatter       
    @property
    def rvir(self):
        if self.ptype == "darkmatter":
            value = self.get_shared_attr(self.ptype, "rvir")
            return value if value is None else value.in_units(self.units['length'])
        raise AttributeError("Attribute 'rvir' is hidden for stars.")

    @rvir.setter
    def rvir(self, value):
        if self.ptype == "darkmatter":
            self.update_shared_attr(self.ptype, "rvir", value)
        else:
            raise AttributeError("Cannot set 'rvir' for stars.")

    @property
    def rs(self):
        if self.ptype == "darkmatter":
            value = self.get_shared_attr(self.ptype, "rs")
            return value if value is None else value.in_units(self.units['length'])
        raise AttributeError("Attribute 'rs' is hidden for stars.")

    @rs.setter
    def rs(self, value):
        if self.ptype == "darkmatter":
            self.update_shared_attr(self.ptype, "rs", value)
        else:
            raise AttributeError("Cannot set 'rs' for stars.")

    @property
    def c(self):
        if self.ptype == "darkmatter":
            value = self.get_shared_attr(self.ptype, "c")
            return value if value is None else value.in_units(self.units['dimensionless'])
        raise AttributeError("Attribute 'c' is hidden for stars.")

    @c.setter
    def c(self, value):
        if self.ptype == "darkmatter":
            self.update_shared_attr(self.ptype, "c", value)
        else:
            raise AttributeError("Cannot set 'c' for stars.")


    @property
    def rockstar_center(self):
        if self.ptype == "darkmatter":
            value = self.get_shared_attr(self.ptype, "rockstar_center")
            return value if value is None else value.in_units(self.units['length'])
        raise AttributeError("Attribute 'rockstar_center' is hidden for stars.")

    @rockstar_center.setter
    def rockstar_center(self, value):
        if self.ptype == "darkmatter":
            self.update_shared_attr(self.ptype, "rockstar_center", value)
        else:
            raise AttributeError("Cannot set 'rockstar_center' for stars.")

    @property
    def rockstar_vel(self):
        if self.ptype == "darkmatter":
            value = self.get_shared_attr(self.ptype, "rockstar_vel")
            return value if value is None else value.in_units(self.units['velocity'])
        raise AttributeError("Attribute 'rockstar_vel' is hidden for stars.")

    @rockstar_vel.setter
    def rockstar_vel(self, value):
        if self.ptype == "darkmatter":
            self.update_shared_attr(self.ptype, "rockstar_vel", value)
        else:
            raise AttributeError("Cannot set 'rockstar_vel' for stars.")
            
    # Dynamically visible properties for stars
    @property
    def ML(self):
        if self.ptype == "stars":
            value = self.get_shared_attr(self.ptype, "ML")
            return value if value is None else value.in_units("Msun/Lsun")
        raise AttributeError("Attribute 'ML' is hidden for dark matter.")

    @ML.setter
    def ML(self, value):
        if self.ptype == "stars":
            self.update_shared_attr(self.ptype, "ML", value)
        else:
            raise AttributeError("Cannot set 'ML' for dark matter.")

    # "Hidden" properties for the partner instance
    @property
    def _rvir(self):
        if self.ptype == "stars":
            value = self.get_shared_attr("darkmatter", "rvir")
            return value if value is None else value.in_units(self.units['length'])
        raise AttributeError("Attribute '_rvir' is not accessible for dark matter.")

    @property
    def _rs(self):
        if self.ptype == "stars":
            value = self.get_shared_attr("darkmatter", "rs")
            return value if value is None else value.in_units(self.units['length'])
        raise AttributeError("Attribute '_rs' is not accessible for dark matter.")

    @property
    def _c(self):
        if self.ptype == "stars":
            value = self.get_shared_attr("darkmatter", "c")
            return value if value is None else value.in_units(self.units['dimensionless'])
        raise AttributeError("Attribute '_c' is not accessible for dark matter.")

    @property
    def _rockstar_center(self):
        if self.ptype == "stars":
            value = self.get_shared_attr("darkmatter", "rockstar_center")
            return value if value is None else value.in_units(self.units['length'])
        raise AttributeError("Attribute '_rockstar_center' is not accessible for dark matter.")

    @property
    def _rockstar_vel(self):
        if self.ptype == "stars":
            value = self.get_shared_attr("darkmatter", "rockstar_vel")
            return value if value is None else value.in_units(self.units['velocity'])
        raise AttributeError("Attribute '_rockstar_vel' is not accessible for dark matter.")

    @property
    def _ML(self):
        if self.ptype == "darkmatter":
            value = self.get_shared_attr("stars", "ML")
            return value if value is None else value.in_units("Msun/Lsun")
        raise AttributeError("Attribute '_ML' is not accessible for stars.")


In [123]:
a = pty("darkmatter", **{
                     'rvir': (30, 'kpc'),
                     'rs': unyt_quantity(4, 'kpc'),
                     'vmax': (50,'km/s')
                 })

{'darkmatter': {'rockstar_center': None, 'rockstar_vel': None, 'rvir': (30, 'kpc'), 'rs': unyt_quantity(4, 'kpc'), 'c': None, 'vmax': (50, 'km/s'), 'vrms': None}, 'stars': {'ML': (3, 'Msun/Lsun')}}


In [125]:
a.rvir

AttributeError: 'tuple' object has no attribute 'in_units'

In [103]:
BaseParticleType._shared_attrs

{'darkmatter': {'rockstar_center': None,
  'rockstar_vel': None,
  'rvir': None,
  'rs': None,
  'c': None,
  'vmax': None,
  'vrms': None},
 'stars': {'ML': None}}

In [104]:
BaseParticleType.set_shared_attrs("stars", 
                                {'ML': (3, 'Msun/Lsun')
                                }
                                )

{'darkmatter': {'rockstar_center': None, 'rockstar_vel': None, 'rvir': None, 'rs': None, 'c': None, 'vmax': None, 'vrms': None}, 'stars': {'ML': (3, 'Msun/Lsun')}}


In [105]:
BaseParticleType._shared_attrs

{'darkmatter': {'rockstar_center': None,
  'rockstar_vel': None,
  'rvir': None,
  'rs': None,
  'c': None,
  'vmax': None,
  'vrms': None},
 'stars': {'ML': (3, 'Msun/Lsun')}}

In [106]:
BaseParticleType.set_shared_attrs("darkmatter", 
                                {
                     'rvir': (30, 'kpc'),
                     'rs': unyt_quantity(4, 'kpc'),
                     'vmax': (50,'km/s')
                 }
                                )

{'darkmatter': {'rockstar_center': None, 'rockstar_vel': None, 'rvir': (30, 'kpc'), 'rs': unyt_quantity(4, 'kpc'), 'c': None, 'vmax': (50, 'km/s'), 'vrms': None}, 'stars': {'ML': (3, 'Msun/Lsun')}}


In [107]:
BaseParticleType._shared_attrs

{'darkmatter': {'rockstar_center': None,
  'rockstar_vel': None,
  'rvir': (30, 'kpc'),
  'rs': unyt_quantity(4, 'kpc'),
  'c': None,
  'vmax': (50, 'km/s'),
  'vrms': None},
 'stars': {'ML': (3, 'Msun/Lsun')}}

In [108]:
BaseParticleType.get_shared_attr("stars", "ML")

(3, 'Msun/Lsun')

In [109]:
BaseParticleType.list_shared_attributes("darkmatter")

['rockstar_center', 'rockstar_vel', 'rvir', 'rs', 'c', 'vmax', 'vrms']